# MONAI REHBERİ
- Bu notebook, MONAI (Medical Open Network for AI) kullanarak tıbbi görüntüler üzerinde sınıflandırma yapmak için gereken temel adımları içeriyor.
- Sırasıyla:
    * Veri setini indirme: MONAI’nin veri seti fonksiyonlarını kullanarak bir medikal görüntü veri seti edineceğiz.
    * Veri ön işleme: Görüntüleri modele uygun hale getirmek için çeşitli dönüşümler uygulayacağız.
    * DenseNet-121 Modeli ile Eğitim: DenseNet-121 modeli kurup eğiteceğiz.
    * Test verisinde sonucun kontrol edilmesi: Eğitilen modeli test verisinde değerlendirip sonuçları gözden geçireceğiz.

- Bu adımda MONAI ve diğer gerekli kütüphaneleri yüklememiz gerekiyor. Aşağıdaki kod, MONAI kütüphanesini ekler:

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[ignite, tqdm]"

- Bu satır, eğer monai yüklü değilse pip install komutunu çalıştırarak MONAI'nin en güncel haftalık sürümünü indirir. Böylece MONAI ile uyumlu ek kütüphaneleri (ignite, tqdm) de kurabiliriz.

## 1. Gerekli Kütüphaneler

- Diğer gerekli kütüphaneleri Import edelim

In [2]:
import logging
import numpy as np
import os
from pathlib import Path
import sys
import tempfile
import torch

from monai.apps import MedNISTDataset
from monai.config import print_config
from monai.data import DataLoader
from monai.engines import SupervisedTrainer
from monai.handlers import StatsHandler
from monai.inferers import SimpleInferer
from monai.networks import eval_mode
from monai.networks.nets import densenet121
from monai.transforms import LoadImageD, EnsureChannelFirstD, ScaleIntensityD, Compose

print_config()

C:\Users\Defuzzy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ignite\handlers\checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer



MONAI version: 1.4.0
Numpy version: 1.26.2
Pytorch version: 2.5.1+cu118
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: C:\Users\<username>\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.5.1
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 10.4.0
Tensorboard version: 2.15.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.20.1+cu118
tqdm version: 4.67.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.7
pandas version: 2.1.4
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow ver

- `print_config()` MONAI'nin sürüm ve konfigürasyon bilgilerini ekrana yazdırır. Böylece sistemin uyumluluğunu kontrol edebiliriz.
- `MedNISTDataset` MONAI’nin sunduğu bir tıbbi görüntü veri setidir. Eğitim ve test işlemlerinde bu veri setini kullanacağız.

## 2. Veri Setini Hazırlama

- Veri setini indirdikten sonra, verileri modele uygun hale getirecek dönüşümleri tanımlıyoruz.

In [3]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

C:\Users\Defuzzy\AppData\Local\Temp\tmpv_73jdoa


- Burada `root_dir` geçici olarak MedNIST veri setini geçici bir yerde depolamamıza olanak tanıyor.
- `Section="training"` ise MedNIST veri seti içinden eğitim verilerini çekmemizi sağlıyor.

- Bu aşamada veriyi dönüştürerek modelimize uygun bir hale getiriyoruz. Yapacağımız her şey veriyi sadece modele hazır hale getirmek için kullanılan yöntemleri içeriyor.
- `LoadImageD` Görüntüyü yükler.
- `EnsureChannelFirstD` Tek kanaldaki görüntülerin formatını dönüştürür.
- `ScaleIntensityD` Görüntü yoğunluğunu yeniden ölçeklendirerek modele uygun hale getirir. Görüntünün kontrastını düzenlediğini söyleyedebiliriz.

In [4]:
transforms = Compose(
    [
        LoadImageD(keys=["image"]),
        EnsureChannelFirstD(keys=["image"]),
        ScaleIntensityD(keys=["image"]),
    ]
)


In [5]:
dataset = MedNISTDataset(root_dir=root_dir, transform=transforms, section="training", download=True)


MedNIST.tar.gz: 59.0MB [00:08, 7.05MB/s]                                                                               


2024-11-19 21:49:35,736 - INFO - Downloaded: C:\Users\Defuzzy\AppData\Local\Temp\tmpv_73jdoa\MedNIST.tar.gz
2024-11-19 21:49:35,822 - INFO - Verified 'MedNIST.tar.gz', md5: 0bc7306e7427e00ad1c5526a6677552d.
2024-11-19 21:49:35,823 - INFO - Writing into directory: C:\Users\Defuzzy\AppData\Local\Temp\tmpv_73jdoa.


Loading dataset: 100%|██████████████████████████████████████████████████████████| 47164/47164 [04:14<00:00, 184.98it/s]


## 3. DenseNet-121 Modeli Tanımlama

- DenseNet-121, sınıflandırma görevlerinde yaygın kullanılan bir modeldir. Burada `in_channels=1` olarak gri tonlamalı görüntüler için ayarlanmıştır.
- `spatial_dims=2` verinin kaç boyutlu bir vektör olduğunu gösterir.
- `in_channels=1` görüntülerin siyah beyaz olduğunu gösterir. RGB (yeşil kırmızı ve mavi kanallar) tipi bir girdi 3 kanala sahip olur.
- `out_channels=6` yapacağımız classification class sayısını gösterir.

In [ ]:
model = densenet121(spatial_dims=2, in_channels=1, out_channels=6).to("cuda:0")

- Büyük veri setleriyle çalışırken bütün Datamızı ramde depolamamız mümkün olmaz. Bundan dolayı veri setini batclere böleriz.
- `Epoch` ise bütün veri setinin baştan sona model tarafından işlenme sayısını söyler.
- `batch_size=64` Her adımda 64 görüntüyü önbelleğe alır.
- `shuffle=True` Her epoch başına veriyi karıştırır, veri sırasını değiştirir.
- Eğitimde Adam optimizasyon algoritması kullanılır ve modelin performansı `CrossEntropyLoss` fonksiyonu ile değerlendirilir.
- `adam optimisation` Bir çok yöntemi bir arada kullanarak daha verimli bir model eğitimi sağlar. Momentum,Adaptive Learning rate ve Bias correction içerir.
- `Cross-Entropy Loss` Modelin ön gördüğü değer ve Gerçek değer arasındaki farkı her örnek için toplar.

In [6]:
max_epochs = 5

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
trainer = SupervisedTrainer(
    device=torch.device("cuda:0"),
    max_epochs=max_epochs,
    train_data_loader=DataLoader(dataset, batch_size=512, shuffle=True, num_workers=4),
    network=model,
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-5),
    loss_function=torch.nn.CrossEntropyLoss(),
    inferer=SimpleInferer(),
    train_handlers=StatsHandler(),
)

## 4. Eğitim ve Değerlendirme

- Modelimizi nihayet eğitmeye başlayabiliriz.

In [7]:
trainer.run()

INFO:ignite.engine.engine.SupervisedTrainer:Engine run resuming from iteration 0, epoch 0 until 5 epochs
2024-11-19 21:56:54,606 - INFO - Epoch: 1/5, Iter: 1/93 -- label: 1.0000 loss: 1.8359 


C:\Users\Defuzzy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\monai\handlers\stats_handler.py:260: UserWarning: ignoring non-scalar output in StatsHandler, make sure `output_transform(engine.state.output)` returns a scalar or dictionary of key and scalar pairs to avoid this warning. image:<class 'monai.data.meta_tensor.MetaTensor'>
  warnings.warn(
C:\Users\Defuzzy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\monai\handlers\stats_handler.py:260: UserWarning: ignoring non-scalar output in StatsHandler, make sure `output_transform(engine.state.output)` returns a scalar or dictionary of key and scalar pairs to avoid this warning. pred:<class 'monai.data.meta_tensor.MetaTensor'>
  warnings.warn(


2024-11-19 21:56:56,296 - INFO - Epoch: 1/5, Iter: 2/93 -- label: 4.0000 loss: 1.8178 
2024-11-19 21:56:57,931 - INFO - Epoch: 1/5, Iter: 3/93 -- label: 4.0000 loss: 1.7837 
2024-11-19 21:56:59,554 - INFO - Epoch: 1/5, Iter: 4/93 -- label: 1.0000 loss: 1.7650 
2024-11-19 21:57:01,184 - INFO - Epoch: 1/5, Iter: 5/93 -- label: 5.0000 loss: 1.7374 
2024-11-19 21:57:02,829 - INFO - Epoch: 1/5, Iter: 6/93 -- label: 0.0000 loss: 1.6787 
2024-11-19 21:57:04,496 - INFO - Epoch: 1/5, Iter: 7/93 -- label: 1.0000 loss: 1.6568 
2024-11-19 21:57:06,117 - INFO - Epoch: 1/5, Iter: 8/93 -- label: 3.0000 loss: 1.6701 
2024-11-19 21:57:07,721 - INFO - Epoch: 1/5, Iter: 9/93 -- label: 1.0000 loss: 1.6119 
2024-11-19 21:57:09,436 - INFO - Epoch: 1/5, Iter: 10/93 -- label: 4.0000 loss: 1.5888 
2024-11-19 21:57:11,067 - INFO - Epoch: 1/5, Iter: 11/93 -- label: 0.0000 loss: 1.5614 
2024-11-19 21:57:12,724 - INFO - Epoch: 1/5, Iter: 12/93 -- label: 2.0000 loss: 1.5311 
2024-11-19 21:57:14,354 - INFO - Epoch: 

- `eval_mode(model)` bloğu, modelin değerlendirme (inference) modunda olduğunu belirtir, bu da dropout ve batch normalization gibi katmanların değerlendirme aşamasında nasıl çalışması gerektiğini belirler.
- `DataLoader`, test veri kümesindeki her bir örneği (bu durumda batch_size=1 olduğundan her seferinde tek bir örnek) yükler.
- `model(item["image"].to("cuda:0"))`, modelin giriş görüntüsünü GPU'ya (cuda:0) taşıyarak tahmin yapmasını sağlar. Sonuçlar daha sonra CPU'ya taşınır ve numpy array'e dönüştürülür.
- `prob.argmax()` ile modelin tahmin ettiği sınıfın olasılıkları arasından en yüksek olanı bulunur ve bu sınıf ismi class_names listesinde karşılık gelen isme dönüştürülür.
- `gt = item["class_name"][0]` ile o örneğin gerçek sınıfı alınır.
- Son olarak, tahmin ve gerçek sınıf ekrana yazdırılır. Bu işlem, max_items_to_print kadar (10 örnek) tekrarlanır.


In [9]:
dataset_dir = Path(root_dir, "MedNIST")
class_names = sorted(f"{x.name}" for x in dataset_dir.iterdir() if x.is_dir())
testdata = MedNISTDataset(root_dir=root_dir, transform=transforms, section="test", download=False, runtime_cache=True)

max_items_to_print = 10
with eval_mode(model):
    for item in DataLoader(testdata, batch_size=1, num_workers=0):
        prob = np.array(model(item["image"].to("cuda:0")).detach().to("cpu"))[0]
        pred = class_names[prob.argmax()]
        gt = item["class_name"][0]
        print(f"Class prediction is {pred}. Ground-truth: {gt}")
        max_items_to_print -= 1
        if max_items_to_print == 0:
            break

Class prediction is AbdomenCT. Ground-truth: AbdomenCT
Class prediction is BreastMRI. Ground-truth: BreastMRI
Class prediction is ChestCT. Ground-truth: ChestCT
Class prediction is CXR. Ground-truth: CXR
Class prediction is Hand. Ground-truth: Hand
Class prediction is HeadCT. Ground-truth: HeadCT
Class prediction is HeadCT. Ground-truth: HeadCT
Class prediction is CXR. Ground-truth: CXR
Class prediction is ChestCT. Ground-truth: ChestCT
Class prediction is BreastMRI. Ground-truth: BreastMRI


- Bu çıktı, modelin test verisindeki örnekler için yaptığı tahminlerle, o örneklerin gerçek etiketlerini (ground-truth) karşılaştırır. Her satır, bir örnek için modelin tahmin ettiği sınıf ile o örneğin doğru sınıfını gösterir.